In [10]:
import fileinput
import re
from nltk.corpus import stopwords

cachedStopWords = stopwords.words("english")

with open('C:\\Users\\IBM_ADMIN\\Desktop\\CaseStudy4\\homeDepRegx.txt', 'r') as file1:
    # read a list of lines into data
    data1 = file1.readlines()

i = 0
for line1 in data1:
    data1[i] = ' '.join([word1.lower() for word1 in line1.split() if word1.lower() not in cachedStopWords]) + "\n"
    #data[i] = big_regex.sub("", line)
    i = i + 1

# and write everything back
with open('C:\\Users\\IBM_ADMIN\\Desktop\\CaseStudy4\\homeDepRegxOutput.txt', 'w') as file2:
    file2.writelines( data1 )

In [11]:
import re, collections

def words(text): return re.findall('[a-z]+', text.lower()) 

def train(features):
    model = collections.defaultdict(lambda: 1)
    for f in features:
        model[f] += 1
    return model

NWORDS = train(words(file('C:\\Users\\IBM_ADMIN\\Desktop\\CaseStudy4\\homeDepRegxOutput.txt').read()))

alphabet = 'abcdefghijklmnopqrstuvwxyz'

def edits1(word):
   splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
   deletes    = [a + b[1:] for a, b in splits if b]
   transposes = [a + b[1] + b[0] + b[2:] for a, b in splits if len(b)>1]
   replaces   = [a + c + b[1:] for a, b in splits for c in alphabet if b]
   inserts    = [a + c + b     for a, b in splits for c in alphabet]
   return set(deletes + transposes + replaces + inserts)

def known_edits2(word):
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1) if e2 in NWORDS)

def known(words): return set(w for w in words if w in NWORDS)

def correct(word):
    candidates = known([word]) or known(edits1(word)) or known_edits2(word) or [word]
    return max(candidates, key=NWORDS.get)

In [1]:
import pandas as pd
df_train = pd.read_csv('C:\\Users\\IBM_ADMIN\\Desktop\\CaseStudy4\\input\\train.csv', encoding="ISO-8859-1")
df_test = pd.read_csv('C:\\Users\\IBM_ADMIN\\Desktop\\CaseStudy4\\input\\test.csv', encoding="ISO-8859-1")
df_attr = pd.read_csv('C:\\Users\\IBM_ADMIN\\Desktop\\CaseStudy4\\input\\attributes.csv')
df_pro_desc = pd.read_csv('C:\\Users\\IBM_ADMIN\\Desktop\\CaseStudy4\\input\\product_descriptions.csv')

In [8]:

print("concat attributes...")
df_attr.dropna(how="all", inplace=True)

# attributes[attributes.product_uid.isnull()]

df_attr["product_uid"] = df_attr["product_uid"].astype(int)
#df_attr["name"] = df_attr["name"].astype(str)
df_attr["value"] = df_attr["value"].astype(str)

def concate_attrs(attrs):
    """
    attrs is all attributes of the same product_uid
    """
    names = attrs["name"]
    values = attrs["value"]
    pairs  = []
    for n, v in zip(names, values):
        pairs.append(' '.join((n, v)))
    return ' '.join(pairs)

product_attrs = df_attr.groupby("product_uid").apply(concate_attrs)

product_attrs = product_attrs.reset_index(name="product_attributes")

concat attributes...


In [14]:
from IPython.display import display
#display(product_attrs)
product_attrs.to_csv("product_attributesBeforeCleaning.csv", encoding="utf-8", index=False)

In [2]:
df_train_test_set = pd.concat((df_train, df_test), axis=0, ignore_index=True)


In [27]:
#df_prodTitle = df_train_test_set['product_uid']
df_prodTitle = pd.DataFrame(data=df_train_test_set['product_uid'])
df_prodTitle['product_title'] = df_train_test_set['product_title']

df_searchTerm = pd.DataFrame(data=df_train_test_set['product_uid'])
df_searchTerm['search_term']= df_train_test_set['search_term']

df_prodTitle.to_csv("df_prodTitle.csv", encoding="utf-8", index=False)
df_searchTerm.to_csv("df_searchTerm.csv", encoding="utf-8", index=False)
print df_prodTitle.shape[0]
print df_searchTerm.shape[0]

240761
240761


In [28]:
df_searchTerm = pd.read_csv('C:\\Users\\IBM_ADMIN\\Desktop\\CaseStudy4\\df_searchTerm.csv')
i = 0
for searchTrm in df_searchTerm.search_term:
    df_searchTerm.search_term[i] = ' '.join([correct(word1.lower()) for word1 in str(searchTrm).split()])
    #data[i] = big_regex.sub("", line)
    i = i + 1 

df_searchTerm.to_csv("df_searchTerm_correctedGrammer.csv", encoding="utf-8", index=False)

In [27]:
print i
print df_searchTerm.shape[0]
print len(df_searchTerm.search_term)
#for i in len(df_searchTerm.search_term):
 #   df_searchTerm.search_term[i] = ' '.join([correct(word1.lower()) for word1 in str(searchTrm).split()])
    #data[i] = big_regex.sub("", line)
    #i = i + 1 
#df_searchTerm.to_csv("df_searchTerm_correctedGrammer.csv", encoding="utf-8", index=False)

1056
240761
240761


In [94]:
from __future__ import division
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')

def correct_spell(str1):
    s = str1.lower()
    #s = s.replace("x", "")
    return ' '.join([correct(word1) for word1 in s.split()])

def str_common_word_prop(str1, str2):
    #str1, str2 = str1.lower(), str2.lower()
    #words, cnt = str1.split(), 0
    #str1length = len(str1)
    #for word in words:
    #    if str2.find(word)>=0:
    #       cnt+=1
    return round(((sum(int(str2.find(word)>=0) for word in str1.split()))/len(str1.split())),2)

def str_stem(str1):
    s = str1.lower()
    s = s.replace("in.","")
    s = s.replace("ft.","")
    s = s.replace("lb.","")
    s = s.replace("sq.","")
    s = s.replace("cu.","")
    s = s.replace("gal.","")
    s = s.replace("oz.","")
    s = s.replace("cm.","")
    s = s.replace("mm.","")
    s = s.replace("amp.","")
    s = s.replace("v.","")
    s = s.replace("w.", "")
    s = s.replace(";"," ")
    s = s.replace(":"," ")
    s = s.replace("&amp;"," ")
    s = s.replace("&amp"," ")
    s = s.replace("?"," ")
    s = s.replace("-"," ")
    s = s.replace("#"," ")
    s = s.replace("("," ")
    s = s.replace(")"," ")
    s = s.replace("//","/")
    s = s.replace(".."," ")
    s = s.replace(" / ","/")
    s = s.replace(" \\ "," ")
    s = s.replace("."," ")
    s = s.replace("*", " ")
    #s = s.replace(" x ", "")
    #s = s.replace("x ", "")
    #s = s.replace(" x", "")
    #s = s.replace("x", "")
    s = ' '.join([stemmer.stem(word1) for word1 in s.split() if word1 not in cachedStopWords])
    #s = (" ").join([stemmer.stem(z) for z in s.split(" ")])
    return s

with open('C:\\Users\\IBM_ADMIN\\Desktop\\CaseStudy4\\stopWords.txt', 'r') as file1:
    # read a list of lines into data
    cachedStopWords = file1.readlines()
        



In [51]:
product_attributesBeforeCleaning = pd.read_csv('C:\\Users\\IBM_ADMIN\\Desktop\\CaseStudy4\\product_attributesBeforeCleaning.csv', encoding="utf-8")
df_prodTitle = pd.read_csv('C:\\Users\\IBM_ADMIN\\Desktop\\CaseStudy4\\df_prodTitle.csv', encoding="utf-8")
df_searchTerm = pd.read_csv('C:\\Users\\IBM_ADMIN\\Desktop\\CaseStudy4\\df_searchTerm_correctedGrammer.csv', encoding="utf-8")
df_pro_desc = pd.read_csv('C:\\Users\\IBM_ADMIN\\Desktop\\CaseStudy4\\input\\product_descriptions.csv', encoding="utf-8")

product_attributesBeforeCleaning['product_attributes'] = product_attributesBeforeCleaning['product_attributes'].map(lambda x:str_stem(x))
df_prodTitle['product_title'] = df_prodTitle['product_title'].map(lambda x:str_stem(x))
df_searchTerm['search_term'] = df_searchTerm['search_term'].map(lambda x:str_stem(x))
df_pro_desc['product_description'] = df_pro_desc['product_description'].map(lambda x:str_stem(x))

In [58]:
import numpy as np
from IPython.display import display
df_train_test_set['product_title'] = df_prodTitle['product_title']
df_train_test_set['search_term'] = df_searchTerm['search_term']
df_train_test_set = pd.merge(df_train_test_set, df_pro_desc, how='left', on='product_uid')
df_train_test_set = pd.merge(df_train_test_set, product_attributesBeforeCleaning, how='left', on='product_uid')

df_train_test_set = df_train_test_set.replace(np.nan,' ', regex=True)
display(df_train_test_set)
df_train_test_set.to_csv("df_fully_processed.csv", encoding="utf-8", index=False)


,id,product_title,product_uid,relevance,search_term,product_description,product_attributes
0,2,simpson strong tie 12 gaug angl,100001,3,angl bracket,"not onli do angl make joint stronger, they als...",bullet01 versatil connector for various 90° co...
1,3,simpson strong tie 12 gaug angl,100001,2.5,l bracket,"not onli do angl make joint stronger, they als...",bullet01 versatil connector for various 90° co...
2,9,behr premium textur deckov 1 sc 141 tugboat wo...,100002,3,deck over,behr premium textur deckov is an innov solid c...,"applic method brush,roller,spray assembl depth..."
3,16,delta vero 1 handl shower onli faucet trim kit...,100005,2.33,rain shower head,updat your bathroom with the delta vero singl ...,bath faucet type combo tub and shower built in...
4,17,delta vero 1 handl shower onli faucet trim kit...,100005,2.67,shower onli faucet,updat your bathroom with the delta vero singl ...,bath faucet type combo tub and shower built in...
5,18,whirlpool 1 9 over the rang convect microwav i...,100006,3,convect otr,achiev delici result is almost effortless with...,applianc type over the rang microwav assembl d...
6,20,whirlpool 1 9 over the rang convect microwav i...,100006,2.67,microwav over stove,achiev delici result is almost effortless with...,applianc type over the rang microwav assembl d...
7,21,whirlpool 1 9 over the rang convect microwav i...,100006,3,microwav,achiev delici result is almost effortless with...,applianc type over the rang microwav assembl d...
8,23,lithonia light quantum 2 light black led emerg...,100007,2.67,emerg light,the quantum adjust 2 light led black emerg lig...,batteri power type ni cad batteri size built i...
9,27,hous of fara 3/4 x 3 x 8 mdf flute case,100009,3,mdf 3/4,get the hous of fara 3/4 x 3 x 8 mdf flute cas...,applic door & window bullet01 made of prime md...


In [57]:
import numpy as np

df_train_test_set.to_csv("df_fully_processed.csv", encoding="utf-8", index=False)


In [95]:
df_train_test_set['product_info'] = df_train_test_set['search_term']+" # "+df_train_test_set['product_title'] +" # "+df_train_test_set['product_description']+" # "+df_train_test_set['product_attributes']
df_train_test_set['prop_in_title'] = df_train_test_set['product_info'].map(lambda x:str_common_word_prop(x.split(' # ')[0],x.split(' # ')[1]))
df_train_test_set['prop_in_description'] = df_train_test_set['product_info'].map(lambda x:str_common_word_prop(x.split(' # ')[0],x.split(' # ')[2]))
df_train_test_set['prop_in_attr'] = df_train_test_set['product_info'].map(lambda x:str_common_word_prop(x.split(' # ')[0],x.split(' # ')[3]))
df_train_test_set['len_of_query'] = df_train_test_set['search_term'].map(lambda x:len(x.split())).astype(np.int64)

In [134]:
#display(df_train_test_set)
df_train_test_set.head()
df_train_test_set.to_csv("df_train_test_set_withFeaturesOnly.csv", encoding="utf-8", index=False)

In [97]:
df_train_test_set.to_csv("df_fully_processed_with_prop.csv", encoding="utf-8", index=False)

In [103]:
num_train = df_train.shape[0]
print num_train
df_train_test_set = df_train_test_set.drop(['search_term','product_title', 'product_attributes','product_description','product_info', 'product_uid' ],axis=1)
df_train_test_set.head()

74067


,id,relevance,prop_in_title,prop_in_description,prop_in_attr,len_of_query
0,2,3,0.50,0.50,0.50,2
1,3,2.5,0.50,0.50,0.50,2
2,9,3,0.50,0.50,1.00,2
3,16,2.33,0.33,0.33,1.00,3
4,17,2.67,1.00,0.67,0.67,3


In [126]:
from sklearn.cross_validation import train_test_split

df_train = df_train_test_set.iloc[:num_train]
print df_train.shape[0]
toSplitTrain, finaltest = train_test_split(df_train, test_size = 0.25)
df_trainsplit, df_test = train_test_split(toSplitTrain, test_size = 0.25)


y_train = df_trainsplit['relevance'].values
X_train = df_trainsplit.drop(['id','relevance'],axis=1).values
y_actual = df_test['relevance'].values
X_test = df_test.drop(['id','relevance'],axis=1).values



74067


In [124]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from nltk.stem.snowball import SnowballStemmer

rf = RandomForestRegressor(n_estimators=15, max_depth=6, random_state=0)
clf = BaggingRegressor(rf, n_estimators=45, max_samples=0.1, random_state=25)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [125]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rms = mean_squared_error(y_actual, y_pred)**0.5
print rms

0.48655337918


In [133]:
y_actual_final = finaltest['relevance'].values
X_test_final = finaltest.drop(['id','relevance'],axis=1).values
y_pred_final = clf.predict(X_test_final)
from sklearn.metrics import mean_squared_error
from math import sqrt

rms = sqrt(mean_squared_error(y_actual_final, y_pred_final))
print rms

0.482713734684


In [121]:
from sklearn import pipeline, grid_search

rfr = RandomForestRegressor()
clf = pipeline.Pipeline([('rfr', rfr)])
param_grid = {'rfr__n_estimators' : list(range(109,121,2)), 'rfr__max_depth': list(range(9,10,2))}
model = grid_search.GridSearchCV(estimator = clf,param_grid = param_grid, n_jobs = 1, cv = 5, verbose = 20)
model.fit(X_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] rfr__n_estimators=109, rfr__max_depth=9 .........................
[CV]  rfr__n_estimators=109, rfr__max_depth=9, score=0.162953 -   2.0s
[CV] rfr__n_estimators=109, rfr__max_depth=9 .........................
[CV]  rfr__n_estimators=109, rfr__max_depth=9, score=0.175247 -   2.0s

[Parallel(n_jobs=1)]: Done   1 tasks       | elapsed:    2.0s
[Parallel(n_jobs=1)]: Done   2 tasks       | elapsed:    4.1s



[CV] rfr__n_estimators=109, rfr__max_depth=9 .........................
[CV]  rfr__n_estimators=109, rfr__max_depth=9, score=0.172374 -   2.0s
[CV] rfr__n_estimators=109, rfr__max_depth=9 .........................
[CV]  rfr__n_estimators=109, rfr__max_depth=9, score=0.157830 -   2.0s

[Parallel(n_jobs=1)]: Done   3 tasks       | elapsed:    6.1s
[Parallel(n_jobs=1)]: Done   4 tasks       | elapsed:    8.2s



[CV] rfr__n_estimators=109, rfr__max_depth=9 .........................
[CV]  rfr__n_estimators=109, rfr__max_depth=9, score=0.166564 -   2.0s
[CV] rfr__n_estimators=111, rfr__max_depth=9 .........................
[CV]  rfr__n_estimators=111, rfr__max_depth=9, score=0.162416 -   2.0s

...

[Parallel(n_jobs=1)]: Done  27 tasks       | elapsed:   58.3s
[Parallel(n_jobs=1)]: Done  28 tasks       | elapsed:  1.0min



[CV] rfr__n_estimators=119, rfr__max_depth=9 .........................
[CV]  rfr__n_estimators=119, rfr__max_depth=9, score=0.157278 -   2.2s
[CV] rfr__n_estimators=119, rfr__max_depth=9 .........................
[CV]  rfr__n_estimators=119, rfr__max_depth=9, score=0.165745 -   2.2s

[Parallel(n_jobs=1)]: Done  29 tasks       | elapsed:  1.0min
[Parallel(n_jobs=1)]: Done  30 tasks       | elapsed:  1.1min


[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  1.1min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(steps=[('rfr', RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'rfr__n_estimators': [109, 111, 113, 115, 117, 119], 'rfr__max_depth': [9]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=20)

In [122]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rms = mean_squared_error(y_actual, y_pred)**0.5
print rms

0.48655337918


In [132]:
y_actual_final = finaltest['relevance'].values
X_test_final = finaltest.drop(['id','relevance'],axis=1).values
y_pred_final = model.predict(X_test_final)
from sklearn.metrics import mean_squared_error
from math import sqrt

rms = sqrt(mean_squared_error(y_actual_final, y_pred_final))
print rms

0.481283520117


In [54]:
s = NaN
s.lower()

NameError: name 'NaN' is not defined